In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras
from keras.layers import LSTM

Using TensorFlow backend.


In [295]:
shift_hrs = 1
shift_steps = shift_hrs*2


data = pd.read_csv('../Data/solar_original.csv')
data['TimeStamp']=pd.to_datetime(data.Date+' '+data.Time,infer_datetime_format=True)
data['DayOfYear']=pd.to_datetime(data.Date,dayfirst=True).dt.dayofyear
data=data.drop(['Date','Time','TimeStamp'],axis=1)
#X = data.iloc[:,1:]
#Y=data['solar_output']

In [311]:
data.head()

,solar_output,temperature,humidity,wind_speed,visibility,serial,DayOfYear
0,0.00000,27.0,74.0,0.0,4.0,1,275
1,0.14216,27.0,74.0,1.9,4.0,2,275
2,1.34680,27.0,74.0,0.0,4.0,3,275
3,3.37716,28.0,70.0,7.4,4.0,4,275
4,5.67044,28.0,70.0,9.3,4.0,5,275


In [307]:
Y=data['solar_output'].shift(-shift_steps)
X=data.drop(['solar_output'],axis=1)

x_data = X.values[0:-shift_steps]
y_data = Y.values[0:-shift_steps]

In [310]:
x_data, y_data

(array([[ 27.  ,  74.  ,   0.  ,   4.  ,   1.  , 275.  ],
        [ 27.  ,  74.  ,   1.9 ,   4.  ,   2.  , 275.  ],
        [ 27.  ,  74.  ,   0.  ,   4.  ,   3.  , 275.  ],
        ...,
        [ 40.  ,   6.  ,  13.  ,   6.  ,  21.  ,  89.  ],
        [ 40.  ,   6.  ,  11.79,   6.  ,  22.  ,  89.  ],
        [ 40.  ,   6.  ,  11.1 ,   6.  ,  23.  ,  89.  ]]),
 array([1.3468 , 3.37716, 5.67044, ..., 7.60269, 4.1613 , 1.59809]))

In [308]:
print (y_data.shape,x_data.shape)

(4498,) (4498, 6)


In [312]:
## Train Test Split

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

x_scaler = MinMaxScaler()
x_data_scaled=x_scaler.fit_transform(x_data)

y_scaler = MinMaxScaler()
y_data_scaled=y_scaler.fit_transform(y_data.reshape(-1, 1)).ravel()

x_train_scaled, x_test_scaled, y_train_scaled, y_test_scaled = train_test_split(x_data_scaled, y_data_scaled, test_size=0.3,random_state=40)

In [313]:

def batch_generator(batch_size, sequence_length):
    """
    Generator function for creating random batches of training-data.
    """

    # Infinite loop.
    while True:
        # Allocate a new array for the batch of input-signals.
        x_shape = (batch_size, sequence_length, 6)
        x_batch = np.zeros(shape=x_shape, dtype=np.float16)

        # Allocate a new array for the batch of output-signals.
        y_shape = (batch_size, sequence_length, 1)
        y_batch = np.zeros(shape=y_shape, dtype=np.float16)

        # Fill the batch with random sequences of data.
        for i in range(batch_size):
            # Get a random start-index.
            # This points somewhere into the training-data.
            idx = np.random.randint(num_train - sequence_length)
            
            # Copy the sequences of data starting at this index.
            x_batch[i] = x_train_scaled[idx:idx+sequence_length]
            y_batch[i] = y_train_scaled[idx:idx+sequence_length]
        
        yield (x_batch, y_batch)

sequence_length=10*24*7 # Days * Iteams per day * No of Days
generator = batch_generator(batch_size=256,
                    sequence_length=sequence_length)

In [314]:
validation_data = (np.expand_dims(x_test_scaled, axis=0),
                   np.expand_dims(y_test_scaled, axis=0))

In [319]:
## [rows, timesteps, columns]
x_train_scaled=x_train_scaled.reshape(x_train_scaled.shape[0],1,x_train_scaled.shape[1])

In [321]:
# design network
n_batch=1
model = keras.models.Sequential()
model.add(keras.layers.LSTM(10, batch_input_shape=(n_batch, x_train_scaled.shape[1], x_train_scaled.shape[2]), stateful=True))
model.add(keras.layers.Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')


In [ ]:
# fit network
n_epoch=100
for i in range(n_epoch):
    model.fit(x_train_scaled, y_train_scaled, epochs=1, batch_size=n_batch, verbose=1, shuffle=False)
    model.reset_states()
# batch forecast
yhat = model.predict(X, batch_size=n_batch)
for i in range(len(y)):
    print('>Expected=%.1f, Predicted=%.1f' % (y[i], yhat[i]))

Epoch 1/1
3148/3148 [==============================] - 36s 11ms/step - loss: 0.0963
Epoch 1/1
3148/3148 [==============================] - 18s 6ms/step - loss: 0.0614
Epoch 1/1
3148/3148 [==============================] - 18s 6ms/step - loss: 0.0416
Epoch 1/1
3148/3148 [==============================] - 18s 6ms/step - loss: 0.0287
Epoch 1/1
3148/3148 [==============================] - 18s 6ms/step - loss: 0.0228
Epoch 1/1
3148/3148 [==============================] - 18s 6ms/step - loss: 0.0202
Epoch 1/1
3148/3148 [==============================] - 18s 6ms/step - loss: 0.0190
Epoch 1/1
3148/3148 [==============================] - 18s 6ms/step - loss: 0.0182
Epoch 1/1
3148/3148 [==============================] - 18s 6ms/step - loss: 0.0176
Epoch 1/1
3148/3148 [==============================] - 18s 6ms/step - loss: 0.0171
Epoch 1/1
3148/3148 [==============================] - 18s 6ms/step - loss: 0.0167
Epoch 1/1
3148/3148 [==============================] - 18s 6ms/step - loss: 0.0164
Epo